In [1]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp311-cp311-linux_x86_64.whl size=825338 sha256=227636fe499e59c3865f6da3609f82e18caaddc145f835f08586e5fd56c90fbc
  Stored in directory: /root/.cache/pip/wheels/b9/0d/8a/0729d2e6e3ca2a898ba55201f905da7db3f838a33df5b3fcdd
Successfully built lightfm


In [2]:
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
from lightfm import LightFM
from lightfm.evaluation import auc_score
from sklearn.model_selection import train_test_split

In [4]:
df=pd.read_csv("/content/merged2.csv")
df.head()

,Unnamed: 0,Unnamed: 0_inventory,Unnamed: 0.1,Product ID,SKU_inventory,Variant ID,Quantity,Color,Tags,Product Type,...,Email,SKU_order,State,Paid at,Prints_order,Neck_order,Sleeves_order,Extracted Tags_order,product_url,url
0,0,20,1910,1948244751137,AB2620PM4423 SANGRIAXS,1938244694336,1,Sangria,"short-sleeve, tops",Tops,...,206344,AB2620PM4423 SANGRIAXS,FL,NaN,Solid,Round Neck,Short Sleeve,Solid Round Neck Short Sleeve,https://www.fashom.com/,https://cdn.shopify.com/s/files/1/2503/4628/pr...
1,1,21,1975,1948244171800,AB22237413 CAMEL/NAVYXS,1938244496106,1,Camel/Navy,"cashmere, fallcatalog, finalsale, long sleeve,...",Tops,...,123946,AB22237413 CAMEL/NAVYXS,MO,NaN,Solid,Turtle Neck,Full Sleeve,Solid Turtle Neck Full Sleeve,https://www.fashom.com/,https://cdn.shopify.com/s/files/1/2503/4628/pr...
2,2,21,1975,1948244171800,AB22237413 CAMEL/NAVYXS,1938244496106,1,Camel/Navy,"cashmere, fallcatalog, finalsale, long sleeve,...",Tops,...,404022,AB22237413 CAMEL/NAVYXS,IL,NaN,Solid,Turtle Neck,Full Sleeve,Solid Turtle Neck Full Sleeve,https://www.fashom.com/,https://cdn.shopify.com/s/files/1/2503/4628/pr...
3,3,21,1975,1948244171800,AB22237413 CAMEL/NAVYXS,1938244496106,1,Camel/Navy,"cashmere, fallcatalog, finalsale, long sleeve,...",Tops,...,159388,AB22237413 CAMEL/NAVYXS,WY,11/9/2022 7:56,Solid,Turtle Neck,Full Sleeve,Solid Turtle Neck Full Sleeve,https://www.fashom.com/,https://cdn.shopify.com/s/files/1/2503/4628/pr...
4,4,21,1975,1948244171800,AB22237413 CAMEL/NAVYXS,1938244496106,1,Camel/Navy,"cashmere, fallcatalog, finalsale, long sleeve,...",Tops,...,306701,AB22237413 CAMEL/NAVYXS,FL,9/14/2022 14:14,Solid,Turtle Neck,Full Sleeve,Solid Turtle Neck Full Sleeve,https://www.fashom.com/,https://cdn.shopify.com/s/files/1/2503/4628/pr...


In [5]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [6]:

def dense_to_sparse(df, user_col='user_id', item_col='product_id', rating_col='purchase',
                 user_mapping=None, item_mapping=None):

    if user_mapping is None:
        user_mapping = {id: i for i, id in enumerate(df[user_col].unique())}
    if item_mapping is None:
        item_mapping = {id: i for i, id in enumerate(df[item_col].unique())}


    # Map user and item IDs to matrix indices
    row = df[user_col].map(user_mapping)
    col = df[item_col].map(item_mapping)
    data = df[rating_col].values
    #Shape of the matrix
    shape = (len(user_mapping), len(item_mapping))
    matrix = coo_matrix((data, (row, col)), shape=shape)
    return  matrix, user_mapping, item_mapping





In [7]:


interaction_matrix, user_mapping, item_mapping = dense_to_sparse(df,"requester_id","Variant ID","Quantity")
interaction_matrix



<1123x99 sparse matrix of type '<class 'numpy.int64'>'
	with 1214 stored elements in COOrdinate format>

In [8]:
train_matrix,_,_=dense_to_sparse(train_df,"requester_id","Variant ID","Quantity",user_mapping=user_mapping,item_mapping=item_mapping)
train_matrix.shape

(1123, 99)

In [9]:
test_matrix,_,_=dense_to_sparse(test_df,"requester_id","Variant ID","Quantity",user_mapping=user_mapping,item_mapping=item_mapping)
test_matrix.shape

(1123, 99)

In [13]:
from lightfm import LightFM
from lightfm.evaluation import auc_score

# Define parameter ranges
component_values = [10, 20, 30, 40, 50]
epoch_values = [10, 20, 30, 40, 50]

# Store results
results = {}

for components in component_values:
    for epochs in epoch_values:
        # Initialize and train model
        model = LightFM(no_components=components, loss='logistic')
        model.fit(train_matrix, epochs=epochs, verbose=False)

        # Evaluate model
        train_auc = auc_score(model, train_matrix).mean()

        # Store result
        results[(components, epochs)] = train_auc
        print(f"Components: {components}, Epochs: {epochs}, AUC: {train_auc:.4f}")

# Display best result
best_params = max(results, key=results.get)
print(f"Best Params: Components={best_params[0]}, Epochs={best_params[1]}, AUC={results[best_params]:.4f}")


Components: 10, Epochs: 10, AUC: 0.7894
Components: 10, Epochs: 20, AUC: 0.7923
Components: 10, Epochs: 30, AUC: 0.7942
Components: 10, Epochs: 40, AUC: 0.7959
Components: 10, Epochs: 50, AUC: 0.7972
Components: 20, Epochs: 10, AUC: 0.7906
Components: 20, Epochs: 20, AUC: 0.7921
Components: 20, Epochs: 30, AUC: 0.7933
Components: 20, Epochs: 40, AUC: 0.7945
Components: 20, Epochs: 50, AUC: 0.7950
Components: 30, Epochs: 10, AUC: 0.7893
Components: 30, Epochs: 20, AUC: 0.7903
Components: 30, Epochs: 30, AUC: 0.7915
Components: 30, Epochs: 40, AUC: 0.7928
Components: 30, Epochs: 50, AUC: 0.7938
Components: 40, Epochs: 10, AUC: 0.7884
Components: 40, Epochs: 20, AUC: 0.7894
Components: 40, Epochs: 30, AUC: 0.7902
Components: 40, Epochs: 40, AUC: 0.7919
Components: 40, Epochs: 50, AUC: 0.7932
Components: 50, Epochs: 10, AUC: 0.7875
Components: 50, Epochs: 20, AUC: 0.7883
Components: 50, Epochs: 30, AUC: 0.7896
Components: 50, Epochs: 40, AUC: 0.7907
Components: 50, Epochs: 50, AUC: 0.7917


In [11]:
test_auc = auc_score(model, test_matrix).mean()


In [12]:
test_auc


#check on the number on epochs and no of components in a for loop

0.75362664